In [11]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import httpx
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.0,api_key=os.environ['OPENAI_API_KEY'],base_url="https://api-inference.modelscope.cn/v1/", http_client=httpx.Client(verify=False),model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")

#### 1. ConversationBufferMemory（对话缓存记忆）
这种记忆允许存储消息，然后从变量中提取消息。

In [12]:
import warnings
warnings.filterwarnings('ignore')
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory = memory,
                                 verbose=True#查看Langchain实际上在做什么，设为FALSE的话只给出回答，看到不到下面绿色的内容
)


In [13]:
conversation.predict(input="Hi, my name is andy")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is andy
AI:

> Finished chain.


'Hello, Andy! How can I assist you today? 😊'

In [14]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is andy
AI: Hello, Andy! How can I assist you today? 😊
Human: What is 1+1?
AI:

> Finished chain.


'The sum of 1 plus 1 is 2. \n\n**Answer:** 1 + 1 equals 2.'

In [15]:
print(memory.buffer)

Human: Hi, my name is andy
AI: Hello, Andy! How can I assist you today? 😊
Human: What is 1+1?
AI: The sum of 1 plus 1 is 2. 

**Answer:** 1 + 1 equals 2.


In [16]:
memory.load_memory_variables({})

{'history': 'Human: Hi, my name is andy\nAI: Hello, Andy! How can I assist you today? 😊\nHuman: What is 1+1?\nAI: The sum of 1 plus 1 is 2. \n\n**Answer:** 1 + 1 equals 2.'}

#### 2. ConversationBufferWindowMemory（对话缓存窗口记忆）
随着对话变得越来越长，所需的内存量也变得非常长。将大量的tokens发送到LLM的成本，也会变得更加昂贵,这也就是为什么API的调用费用，通常是基于它需要处理的tokens数量而收费的。

In [18]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=1)     # k=1表明只保留一个对话记忆    
conversation = ConversationChain(llm=llm, memory = memory,
    verbose=True    #这里改为FALSE不显示提示，你可以尝试修改为TRUE后的结果
)

In [19]:
conversation.predict(input="Hi, my name is andy")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is andy
AI:

> Finished chain.


'Hello, Andy! How can I assist you today?'

In [20]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is andy
AI: Hello, Andy! How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The sum of 1 and 1 is 2. \n\n**Step-by-Step Explanation:**\n\n1. Start with the number 1.\n2. Add another 1 to it.\n3. The total is 2.\n\nThus, 1 + 1 equals 2.'

In [21]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: The sum of 1 and 1 is 2. 

**Step-by-Step Explanation:**

1. Start with the number 1.
2. Add another 1 to it.
3. The total is 2.

Thus, 1 + 1 equals 2.
Human: What is my name?
AI:

> Finished chain.


"I don't have a name, but I'm here to help you with whatever you need!"